<a href="https://colab.research.google.com/github/shivamsri07/vectors_and_llms/blob/main/annoy_algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import numpy as np

def get_hyperplane(p1, p2):
    """
    Creates a hyperplane defined by the normal vector (p1 - p2)
    and a midpoint. For simplicity, we can just use the normal
    and a dot product against the midpoint to determine side.
    """
    normal = p1 - p2
    midpoint = (p1 + p2) / 2
    # For a point x, side is determined by np.dot(x - midpoint, normal)
    return normal, midpoint

def conceptual_annoy_split_node(points_indices, all_data):
    """
    Conceptually splits a node in an ANNOY tree.
    points_indices: list of indices of points in this node.
    all_data: the full dataset (e.g., a list or np.array of vectors).
    """
    if len(points_indices) < 2: # Cannot pick two distinct points
        return None, None, points_indices # Leaf node or too few points

    # 1. Pick two random points from the current node's points
    idx1, idx2 = np.random.choice(points_indices, 2, replace=False)
    p1 = all_data[idx1]
    p2 = all_data[idx2]

    # 2. Create a hyperplane between them
    normal, midpoint = get_hyperplane(p1, p2)

    left_child_indices = []
    right_child_indices = []

    # 3. Divide other points
    for idx in points_indices:
        if idx == idx1 or idx == idx2: # Points defining the plane might be handled based on implementation
            continue
        point = all_data[idx]
        # Determine which side of the hyperplane the point lies on
        # np.dot(point - midpoint, normal) > 0 means one side, <= 0 means the other
        if np.dot(point - midpoint, normal) > 0:
            left_child_indices.append(idx)
        else:
            right_child_indices.append(idx)

    # In a real implementation, the points p1 and p2 might also be assigned
    # to one side or handled such that they don't get lost.
    # For simplicity, they are just used to define the split here.
    # Often, one point goes left, the other goes right, or they are added to both/neither
    # and the remaining points are split. Let's put p1 in left, p2 in right for this concept.
    # A robust way is to ensure all points are assigned.
    # A simple assignment after initial split:
    # if np.dot(p1 - midpoint, normal) > 0: left_child_indices.append(idx1)
    # else: right_child_indices.append(idx1)
    # if np.dot(p2 - midpoint, normal) > 0: left_child_indices.append(idx2) # This is just illustrative.
    # else: right_child_indices.append(idx2)


    # This is a highly simplified split. Real ANNOY handles edge cases and details more robustly.
    # The actual split point is the hyperplane itself, not a value.
    print(f"Splitting with hyperplane from points {idx1} and {idx2}")
    print(f"  Normal vector: {normal}")
    print(f"  Midpoint: {midpoint}")
    print(f"  Left child gets {len(left_child_indices)} points, Right child gets {len(right_child_indices)} points")

    return left_child_indices, right_child_indices, None # No points left if all are distributed

# This conceptual function would be called recursively to build a tree.
# A forest would involve calling this tree-building process multiple times.

In [9]:
# Let's create a dummy vector set and build a simple conceptual tree layer.
dummy_vectors = np.array([
    [1, 1],
    [2, 2],
    [3, 3],
    [4, 4],
    [5, 5],
    [6, 6],
    [7, 7],
    [8, 8],
    [9, 9],
    [10, 10]
])

# Indices of all points initially in the root node
all_indices = list(range(len(dummy_vectors)))

# Perform a single conceptual split on the root node
left_indices, right_indices, remaining_indices = conceptual_annoy_split_node(all_indices, dummy_vectors)

print("\nAfter first split:")
print(f"Left child indices: {left_indices}")
print(f"Right child indices: {right_indices}")
print(f"Remaining indices (should be None in this concept): {remaining_indices}")

# In a real tree, you would recursively call conceptual_annoy_split_node
# on `left_indices` and `right_indices` until nodes are small enough (leaf nodes).

# Example of a recursive call (conceptual, not full tree build):
if left_indices is not None and len(left_indices) > 1:
    print("\nConceptually splitting the left child:")
    left_left, left_right, _ = conceptual_annoy_split_node(left_indices, dummy_vectors)
    print(f"  Left-left indices: {left_left}")
    print(f"  Left-right indices: {left_right}")

if right_indices is not None and len(right_indices) > 1:
    print("\nConceptually splitting the right child:")
    right_left, right_right, _ = conceptual_annoy_split_node(right_indices, dummy_vectors)
    print(f"  Right-left indices: {right_left}")
    print(f"  Right-right indices: {right_right}")

Splitting with hyperplane from points 2 and 6
  Normal vector: [-4 -4]
  Midpoint: [5. 5.]
  Left child gets 3 points, Right child gets 5 points

After first split:
Left child indices: [0, 1, 3]
Right child indices: [4, 5, 7, 8, 9]
Remaining indices (should be None in this concept): None

Conceptually splitting the left child:
Splitting with hyperplane from points 0 and 3
  Normal vector: [-3 -3]
  Midpoint: [2.5 2.5]
  Left child gets 1 points, Right child gets 0 points
  Left-left indices: [1]
  Left-right indices: []

Conceptually splitting the right child:
Splitting with hyperplane from points 9 and 4
  Normal vector: [5 5]
  Midpoint: [7.5 7.5]
  Left child gets 2 points, Right child gets 1 points
  Right-left indices: [7, 8]
  Right-right indices: [5]
